### Классификация. Логистическая регрессия

In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [4]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

##### 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [15]:
def calc_logloss(y, y_pred):
    pred_log1 = np.array([0 if x == 0 else np.log(x) for x in y_pred])
    pred_log2 = np.array([0 if x == 1 else np.log(1.0 - x) for x in y_pred])
    err = - np.mean(y * pred_log1 + (1.0 - y) * pred_log2)
    return err

In [16]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

##### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным

In [19]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [20]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)n
    return W

In [80]:
W = eval_model(X_st, y, iterations=100000, eta=1e-1)

0 [ 0.45877546 -0.2833519   0.6449505   1.46641523] 1.1785958344356262
10000 [-10.74518967  -1.38402525  -2.42101957   9.13461388] 0.2321387488434184
20000 [-15.42754617  -1.78141273  -3.82881206  12.91762302] 0.19343207615808586
30000 [-19.03214673  -2.10018205  -4.88462075  15.84722845] 0.1705495036750294
40000 [-21.99898567  -2.36617547  -5.74184132  18.25923693] 0.15508406003773303
50000 [-24.52678352  -2.59425824  -6.4658027   20.31294088] 0.14388003971456603
60000 [-26.7341771   -2.79398809  -7.09424451  22.10455082] 0.13535136053673655
70000 [-28.69854717  -2.9718268   -7.6512386   23.6970608 ] 0.12860785839125297
80000 [-30.4730121   -3.13231139  -8.1530329   25.13378696] 0.12311312476808871
90000 [-32.09545327  -3.27872995  -8.61106042  26.44564896] 0.1185257929395436


##### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [81]:
def calc_pred_proba(W, X):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [82]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([3.76929484e-01, 1.43636180e-02, 9.99999995e-01, 1.10343509e-07,
       9.32592337e-01, 6.42801817e-02, 1.00000000e+00, 2.06617132e-02,
       6.05609842e-01, 9.99999456e-01])

##### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [83]:
def calc_pred(W, X):
    return np.array([1 if x > 0.5 else 0 for x in calc_pred_proba(W, X)])

In [84]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

##### 5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score

In [85]:
def assessment_classification_quality(y, y_pred):
    accuracy = np.sum([1 if t[0] == t[1] else 0 for t in zip(y, y_pred)]) / len(y)
    print(f'accuracy = {accuracy}')
    
    tp = sum([1 if t[0] == t[1] == 1 else 0 for t in zip(y, y_pred)])
    tn = sum([1 if t[0] == t[1] == 0 else 0 for t in zip(y, y_pred)])
    fp = sum([1 if t[0] == 1 and t[1] == 0 else 0 for t in zip(y, y_pred)])
    fn = sum([1 if t[0] == 0 and t[1] == 1 else 0 for t in zip(y, y_pred)])
    print('Confustion matrix')
    print('\t1\t0')
    print(f'1\t{tp}\t{fp}')
    print(f'0\t{fn}\t{tn}')
    
    precision = tp / (tp + fp)
    print(f'precision = {precision}')
    
    recall = tp / (tp + fn)
    print(f'recall = {recall}')
    
    F = 2 * precision * recall / (precision + recall)
    print(f'F = {F}')

In [86]:
assessment_classification_quality(y, y_pred)

accuracy = 1.0
Confustion matrix
	1	0
1	5	0
0	0	5
precision = 1.0
recall = 1.0
F = 1.0


##### 6. Могла ли модель переобучиться? Почему?

Модель могла переобучиться, так как в датасете слишком мало наблюдений